# Encriptación y Desencriptación RSA con Sensores en CUDA y C++
---
<p> Universidad del Valle de Guatemala | Facultad de Ingeniería | CC3086 Programación de Microprocesadores <p>

><p>José Pablo Kiesling Lange | Carné 21581</p>
><p>Adrián Ricardo Flores Trujillo | Carné 21500 </p>
><p>Andrea Ximena Ramírez Recinos | Carné 21874</p>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-fvvidpos
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-fvvidpos


In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


## Apartado de Encriptación RSA
---
<p align = "Justify"> Este es un sistema criptográfico de llave pública. En el sistema RSA, cada participante hace pública una llave para cifrar y oculta la llave para descifrar.

El receptor elige dos números primos p y q y calcula n = pq.
Luego, el receptor calcula </p>

```python
  φ = (p-1)(q-1) 
```

y se elige un entero e tal que 

      mcd(e, φ) = 1.

<p align = "Justify"> El mensaje se cifra convirtiendo las letras a numeros. Para este proyecto se utilizara A = 1, B = 2, C = 3, etc. El espacio en blanco es cero y todos los demás caracteres son numeros negativos.</p> <br>

<p align = "Justify"> Luego se concatenan estos números de dos dígitos en bloques de tamaño 2N, en donde 2N es el mayor entero par tal que el número 2525...25 con 2N dígitos no exceda n.</p> <br>

<p align = "Justify"> Para encriptar se transforma cada bloque m a un bloque cifrado c de la siguiente manera:</p> 

```python
  c = pow(m,e) % (n)
```

In [ ]:
%%cu

#include <iostream>
#include <math.h>
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <sstream>
#include <string>
#include <fstream>
#include <bits/stdc++.h>

//Solo para PRUEBA;
#define thread_Number 30

using namespace std;

int powerMod(int a, int b, int m)
{
    int res = 1;
    a = a % m;
    for(int i=1 ; i<=b ; ++i)
        res = (res * a) % m;
    
    return res;
}

void divideString(string str, int n, string* res) //From: https://www.geeksforgeeks.org/divide-a-string-in-n-equal-parts/
{
    if (str.length() % n != 0) {
        cout << "Invalid Input: String size";
        cout << " is not divisible by n";
        return;
    }
 
    int parts = str.length() / n;
 
    for(int i=0; i<parts; i++) {
        res[i] = str.substr(i*n, n);
    }
}

__global__ void findPrime(int* num, int n){
    int i = threadIdx.x + blockDim.x * blockIdx.x;
    int k,l;
    bool flag = true;
 
    // Check from 2 to square root of n
    for (int j = 2; j <= sqrt((float)num[i]); j++)
        if (num[i] % j == 0)
            flag = false;

    if (!flag){
        if (num[i] & 1)
            num[i] += 2;
        else
            num[i]++;
    
        for (k = num[i]; k <= n; k += 2) {
            if (k % 2 == 0)
                continue;
            for (l = 3; l <= sqrt((float)k); l += 2) {
                if (k % l == 0)
                    break;
            }
            if (l > sqrt((float)k))
                break;
        }
        num[i] = k;
    }
    
}

string block_calculation(int n_forEncryption) {
    string max_block_digits = "26";

    while(n_forEncryption > stoi(max_block_digits +"26")){
       max_block_digits+="26";
    }
    return max_block_digits;
}

int main() {
    string message_toEncrypt = "";

    FILE* inputFile;
    char    *text;
    long    numbytes;
    inputFile = fopen("/content/drive/MyDrive/datos.txt", "r");

    if(inputFile == NULL){
        cout << "Error opening the file" << endl;
    }

    fseek(inputFile, 0L, SEEK_END);
    numbytes = ftell(inputFile);
    fseek(inputFile, 0L, SEEK_SET);	

    text = (char*)calloc(numbytes, sizeof(char));	
    if(text == NULL)
        return 1;

    fread(text, sizeof(char), numbytes, inputFile);
    fclose(inputFile);

    string fileText;
    int p_primeForEncryption;
    int q_primeForEncryption;

    int cont;
    for (int i = 0; i < numbytes; i++) {
      if(text[i] == ',' && message_toEncrypt == ""){
          cont = i;
          message_toEncrypt = fileText;
      } else if(text[i] == ','){
          p_primeForEncryption = stoi( fileText.substr(cont+1, 2) );
      } else if (i == numbytes - 1){
          fileText += text[i];
          q_primeForEncryption = stoi( fileText.substr(cont+4, 2) );
      }

      fileText += text[i];

    }

    int n_forEncryption;
    int e_coprimeForEncryption = 17;

    //Host memory allocation of the array containing primes p and q
    int *h_pq = (int*)malloc(2*sizeof(int));

    //Device memory allocation of the array containing primes p and q
    int *d_pq;
    cudaMalloc((void**)&d_pq, 2*sizeof(int));

    while(true){
      //Lectura de p y q con los sensores

      n_forEncryption = (p_primeForEncryption)*(q_primeForEncryption);

      //Initializing data of the array containing primes p and q
      h_pq[0] = p_primeForEncryption;
      h_pq[1] = q_primeForEncryption;

      cudaMemcpy(d_pq, h_pq, 2*sizeof(int), cudaMemcpyHostToDevice);

      //Caling the kernel to return nearest prime
      findPrime<<<1,2>>>(d_pq, n_forEncryption);

      cudaMemcpy(h_pq, d_pq, 2*sizeof(int), cudaMemcpyDeviceToHost );

      p_primeForEncryption = h_pq[0];
      q_primeForEncryption = h_pq[1];

      n_forEncryption = (p_primeForEncryption)*(q_primeForEncryption);

      if (n_forEncryption > 2626)
        break;
      
      p_primeForEncryption++;
      q_primeForEncryption++;
    }

    //Declaration of Public Key 
    int PublicKey[] = {n_forEncryption,e_coprimeForEncryption};

    //Calculating blocks to encrypt the message
    string block_calculation_result = block_calculation(n_forEncryption);
    int characters_to_break_into = block_calculation_result.length();

    //Encoding using ASCII
    string message_toEncrypt_Numbers = "";
    transform(message_toEncrypt.begin(), message_toEncrypt.end(),message_toEncrypt.begin(), ::toupper);

    for(int i=0;i<message_toEncrypt.length();i++){
        if(((int) message_toEncrypt[i] - 64)< 10 && ((int) message_toEncrypt[i] - 64) >= 1){
            message_toEncrypt_Numbers += "0"+ to_string((int) message_toEncrypt[i] - 64);
        }
        else if(((int) message_toEncrypt[i] - 64)< 1){
            message_toEncrypt_Numbers += "00";
        }
        else{
            message_toEncrypt_Numbers += to_string((int) message_toEncrypt[i] - 64);
        }
    }

    int temp = message_toEncrypt_Numbers.length()/characters_to_break_into;
    string partsOfMessage_inNumbers[temp];
    divideString(message_toEncrypt_Numbers, characters_to_break_into, partsOfMessage_inNumbers);

    printf("Mensaje final encriptado: ");
    for (int i = 0; i < temp; i++) {
        string exp = to_string(powerMod(stoi(partsOfMessage_inNumbers[i]), e_coprimeForEncryption, n_forEncryption));
        while(exp.length() != characters_to_break_into){
            exp = "0" + exp;
        }
        
        partsOfMessage_inNumbers[i] = exp;
        cout << partsOfMessage_inNumbers[i] << " ";
    }

    string out = "";
    for (int i = 0; i < temp; i++) {
        out += partsOfMessage_inNumbers[i] + " ";
    }

    inputFile = fopen("/content/drive/MyDrive/datos.txt", "w");

    fwrite(out.substr(0,out.length()-1).c_str(), sizeof(char), out.length(), inputFile);

    fclose(inputFile);

    return 0;

}

Mensaje final encriptado: 4198 0462 5593 3632 2882 0694 0710 4554 3499 2919 1124 6853 0001 2767 5616 3093 


## Apartado de Desencriptación RSA
---
<p align = "Justify"> Para desencriptar se debe de calcular el entero 0 < d < $, que satisface</p>

      d·e ≡ 1 (mod φ)

<p align = "Justify">El número d se guarda en secreto y se usa para descifrar los mensajes; esta es la llave privada. Luego, para desencriptar, se transforma cada bloque de cifrado c a un bloque decifrado m de la siguiente manera: </p>

```python
    m = pow(c,d) % (n)
```



In [ ]:
%%cu

#include <iostream>
#include <math.h>
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <sstream>
#include <string>
#include <fstream>
#include <bits/stdc++.h>

//Solo para PRUEBA;
#define thread_Number 30

using namespace std;

void divideString(string str, int n, string* res) //From: https://www.geeksforgeeks.org/divide-a-string-in-n-equal-parts/
{
    if (str.length() % n != 0) {
        cout << "Invalid Input: String size";
        cout << " is not divisible by n";
        return;
    }
 
    int parts = str.length() / n;
 
    for(int i=0; i<parts; i++) {
        res[i] = str.substr(i*n, n);
    }
}

__global__ void findPrime(int* num, int n){
    int i = threadIdx.x + blockDim.x * blockIdx.x;
    int k,l;
    bool flag = true;
 
    // Check from 2 to square root of n
    for (int j = 2; j <= sqrt((float)num[i]); j++)
        if (num[i] % j == 0)
            flag = false;

    if (!flag){
        if (num[i] & 1)
            num[i] += 2;
        else
            num[i]++;
    
        for (k = num[i]; k <= n; k += 2) {
            if (k % 2 == 0)
                continue;
            for (l = 3; l <= sqrt((float)k); l += 2) {
                if (k % l == 0)
                    break;
            }
            if (l > sqrt((float)k))
                break;
        }
        num[i] = k;
    }
    
}

// First kernel to calculate d value for the encryption
__global__ void calculatingDForEncryption(int *d_ArrayOfValuesForD, int *d_dValue, int phi_ForEncryption, int e_coprimeForEncryption)
{
  	int myID = threadIdx.x + blockDim.x * blockIdx.x;
    int temp = (d_ArrayOfValuesForD[myID]*e_coprimeForEncryption) % phi_ForEncryption;

    if(temp == 1){
        *d_dValue = d_ArrayOfValuesForD[myID];
    }
}


//Second kernel to decrypt the message
__global__ void decrypt(int* d_arrayOfValues, int *d_dValue, int n_forEncryption){
    int myID = threadIdx.x + blockDim.x * blockIdx.x;
    int res = 1;
    d_arrayOfValues[myID] = d_arrayOfValues[myID] % n_forEncryption;

    for(int i=1 ; i<=*d_dValue ; ++i)
        res = (res * d_arrayOfValues[myID]) % n_forEncryption;
    d_arrayOfValues[myID] = res;
}

string block_calculation(int n_forEncryption) {
    string max_block_digits = "26";

    while(n_forEncryption > stoi(max_block_digits +"26")){
       max_block_digits+="26";
    }
    return max_block_digits;
}

int main(){
    string message_toDecrypt;
    int p_primeForEncryption;
    int q_primeForEncryption;

    FILE* inputFile;
    char    *text;
    long    numbytes;
    inputFile = fopen("/content/drive/MyDrive/datos.txt", "r");

    if(inputFile == NULL){
        cout << "Error opening the file" << endl;
    }

    fseek(inputFile, 0L, SEEK_END);
    numbytes = ftell(inputFile);
    fseek(inputFile, 0L, SEEK_SET);	

    text = (char*)calloc(numbytes, sizeof(char));	
    if(text == NULL)
        return 1;

    fread(text, sizeof(char), numbytes, inputFile);
    fclose(inputFile);

    string fileText;

    int cont;
    for (int i = 0; i < numbytes; i++) {
      if(text[i] == ',' && message_toDecrypt == ""){
          cont = i;
          message_toDecrypt = fileText;
      } else if(text[i] == ','){
          p_primeForEncryption = stoi( fileText.substr(cont+1, 2) );
      } else if (i == numbytes - 1){
          fileText += text[i];
          q_primeForEncryption = stoi( fileText.substr(cont+4, 2) );
      }

      fileText += text[i];

    }


    int e_coprimeForEncryption = 17;

    int n_forEncryption;

    //Host memory allocation of the array containing primes p and q
    int *h_pq = (int*)malloc(2*sizeof(int));

    //Device memory allocation of the array containing primes p and q
    int *d_pq;
    cudaMalloc((void**)&d_pq, 2*sizeof(int));

    while(true){
      //Lectura de p y q con los sensores

      n_forEncryption = (p_primeForEncryption)*(q_primeForEncryption);

      //Initializing data of the array containing primes p and q
      h_pq[0] = p_primeForEncryption;
      h_pq[1] = q_primeForEncryption;

      cudaMemcpy(d_pq, h_pq, 2*sizeof(int), cudaMemcpyHostToDevice);

      //Caling the kernel to return nearest prime
      findPrime<<<1,2>>>(d_pq, n_forEncryption);

      cudaMemcpy(h_pq, d_pq, 2*sizeof(int), cudaMemcpyDeviceToHost );

      p_primeForEncryption = h_pq[0];
      q_primeForEncryption = h_pq[1];

      n_forEncryption = (p_primeForEncryption)*(q_primeForEncryption);

      if (n_forEncryption > 2626)
        break;
      
      p_primeForEncryption++;
      q_primeForEncryption++;
    }

    int phi_ForEncryption = (p_primeForEncryption - 1)*(q_primeForEncryption - 1);
    size_t sizeOfPhi = phi_ForEncryption * sizeof(int);

    ///Memory allocation of values for d in host
    int *h_ArrayOfValuesForD = (int*)malloc(sizeOfPhi);
    int h_dValue;

    //Delclaring the device data (GPU)
    int *d_ArrayOfValuesForD, *d_dValue;

    //Allocate memory in the device (GPU)
	  cudaMalloc((void**)&d_ArrayOfValuesForD, sizeOfPhi);
    cudaMalloc((void**)&d_dValue,sizeof(int));

    //Initialize array data for the host (CPU) (For values of d)
    for(int i=0; i<=phi_ForEncryption; i++){
        h_ArrayOfValuesForD[i] = i;
    }

    //Copying data from host to device
    cudaMemcpy(d_ArrayOfValuesForD, h_ArrayOfValuesForD, sizeOfPhi, cudaMemcpyHostToDevice);

    //Declaration of blocks to use
    int blocksPerGrid = (phi_ForEncryption + thread_Number - 1) /thread_Number;

    //Launching the kernel to calculate d_forEncryption
    calculatingDForEncryption<<<thread_Number,blocksPerGrid>>>(d_ArrayOfValuesForD,d_dValue,phi_ForEncryption,e_coprimeForEncryption);

    //Sending back result to host
    cudaMemcpy(&h_dValue, d_dValue, sizeof(int), cudaMemcpyDeviceToHost);

    int block_size = block_calculation(n_forEncryption).length();
    int temporalValue = message_toDecrypt.length()/block_size;

    //Memory allocation for the array of strings in the host
    int *blocks = (int*)malloc(temporalValue*sizeof(int));

    //Delclaring the device data (GPU)
    int *d_blocks;

    for(int i = 0; i<temporalValue; i++){
        //cout<<blocks[i]<<endl;
        if(i == 0){
            blocks[i] = stoi(message_toDecrypt.substr(i*block_size, block_size));
        } else {
            try{
                blocks[i] = stoi(message_toDecrypt.substr((i*block_size)+i, block_size));
            } catch(...){}
        }
    }

    //Allocate memory in the device (GPU)
	  cudaMalloc((void**)&d_blocks, temporalValue*sizeof(int));

    //Copying data from host to device
    cudaMemcpy(d_blocks, blocks, temporalValue*sizeof(int), cudaMemcpyHostToDevice);

    //Calling the kernel to decrypt
    decrypt<<<1,temporalValue>>>(d_blocks,d_dValue,n_forEncryption);
    cudaMemcpy(blocks,d_blocks, temporalValue*sizeof(int), cudaMemcpyDeviceToHost);

    string temporalArray[temporalValue];

    for(int i  = 0; i < temporalValue; i++){
        //cout << blocks[i] << endl;
    }

    for(int i=0;i<temporalValue;i++){
      temporalArray[i] = to_string(blocks[i]);
    }

    for(int i=0;i<temporalValue;i++){
      while(temporalArray[i].length() != block_size){
        temporalArray[i] = "0" + temporalArray[i];
      }
    }

    string temporalArrayStrings [block_size/2];
    string finalResultString = "";

    for(int i=0;i<temporalValue;i++){
        divideString(temporalArray[i],block_size/2,temporalArrayStrings);
        int tempChar = stoi(temporalArrayStrings[0])+64;
        if(tempChar<=90 && tempChar>64){
            finalResultString = finalResultString + (char)(stoi(temporalArrayStrings[0])+64);
        }
        else if(tempChar<65){
             finalResultString += " ";
        }
        tempChar = stoi(temporalArrayStrings[1])+64;
        if(tempChar<=90 && tempChar>64){
            finalResultString = finalResultString + (char)(stoi(temporalArrayStrings[1])+64);
        }

        else if(tempChar<65){
            finalResultString += " ";
        }
    }

    cout<< "Mensaje desencriptado: " << finalResultString;

    return 0;
}

Mensaje desencriptado: BUENOS DIAS MI NOMBRE ES ANDREA       
